<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-BFGS-to-obtain-MAP-estimate" data-toc-modified-id="Using-BFGS-to-obtain-MAP-estimate-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using BFGS to obtain MAP estimate</a></span><ul class="toc-item"><li><span><a href="#Define-Functions" data-toc-modified-id="Define-Functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define Functions</a></span></li><li><span><a href="#Generate-Data" data-toc-modified-id="Generate-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate Data</a></span></li></ul></li><li><span><a href="#Run-BFGS-on-Koenecker-Data" data-toc-modified-id="Run-BFGS-on-Koenecker-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run BFGS on Koenecker Data</a></span></li><li><span><a href="#Criticize-Results" data-toc-modified-id="Criticize-Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Criticize Results</a></span></li></ul></div>

# Using BFGS to obtain MAP estimate

In [1]:
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed

import tensorflow as tf
import numpy as np
import math
import pandas as pd
import tqdm
from tqdm import tqdm_notebook

In [2]:
sess = tf.Session()

In [3]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

In [4]:
np_cascades = np.ones((len(v),numNodes),np.float32)*10
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]
        
casc = np_cascades[0]

In [5]:
def cascadeLogProb(time, seed):
    np.where()
    
    
    # Store number of nodes
    n = time.shape[0]

    # Transpose times and reduce minimum
    times_T = tf.minimum(tf.transpose(time),T)

    # Initialize transmission times to be max time except for seed node
    transmission = tf.ones(n)*T
    transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

    
    # Continually update transmissions
    for _ in range(n):

        # Tile transmission
        transmission_tiled = tf.reshape(tf.tile(transmission,[n]),[n,n])

        # Add transposed times and tiled transmissions
        potential_transmission = tf.add(transmission_tiled,times_T)

        # Find minimum path from all new 
        potential_transmission_row = tf.reduce_min(potential_transmission, reduction_indices=[1])

        # Concatenate previous transmission and potential new transmission
        potential_transmission_stack = tf.stack([transmission,potential_transmission_row],axis=0)

        # Take the minimum of the original transmission and the potential new transmission
        transmission = tf.reduce_min(potential_transmission_stack, reduction_indices=[0])

    return transmission

## Define Functions

In [6]:
def build_cascade(time, seed, T):
    # Store number of nodes
    n = time.shape[0]

    # Transpose times and reduce minimum
    times_T = tf.minimum(tf.transpose(time),T)

    # Initialize transmission times to be max time except for seed node
    transmission = tf.ones(n)*T
    transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

    
    # Continually update transmissions
    for _ in range(n):

        # Tile transmission
        transmission_tiled = tf.reshape(tf.tile(transmission,[n]),[n,n])

        # Add transposed times and tiled transmissions
        potential_transmission = tf.add(transmission_tiled,times_T)

        # Find minimum path from all new 
        potential_transmission_row = tf.reduce_min(potential_transmission, reduction_indices=[1])

        # Concatenate previous transmission and potential new transmission
        potential_transmission_stack = tf.stack([transmission,potential_transmission_row],axis=0)

        # Take the minimum of the original transmission and the potential new transmission
        transmission = tf.reduce_min(potential_transmission_stack, reduction_indices=[0])

    return transmission

## Generate Data

In [7]:
alpha_0 = np.array([[0, 1, 0, 0, .5, 0, 0, 0, 0, 0],
                    [0, 0, 1, .3, 0, 0, 2, .5, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0, .2, 0, 0],
                    [0, 0, 0, 0, 1, 0, .1, 0, .5, 0],
                    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0, .2, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.float32)

alpha_1 = np.array([[0, 1, 0, .3, .8, 0, 0, 1.2, 0, 0],
                    [0, 0, 1.4, .3, 0, 0, 3.5, .6, 0, 0],
                    [0, 0, 0, .5, 0, 0, 0, .3, 0, 0],
                    [0, 0, 0, 0, .5, 0, .7, 0, .4, 0],
                    [0, 0, 0, 0, 0, .8, 0, 0, 1.5, 0],
                    [0, 0, 0, 0, 0, 0, 1.4, 0, .6, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1.1, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1.7, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, .2],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.float32)

alpha = np.stack((alpha_0,alpha_1))

In [26]:
import random
def sampleCascade(alpha, T, n):
    alpha_tf = tf.convert_to_tensor(alpha, dtype=tf.float32)
    nodes = alpha.shape[1]
    
    np_topics = np.zeros((n,2))
    
    
    cascade=[]
    topic=[]
    for i in tqdm_notebook(range(n)):
        topic_pos = random.randint(0,1)
        np_topics[i,topic_pos]=1
        
        tau = ed.Exponential(tf.gather(alpha_tf,indices=topic_pos))
        seed = random.randint(0,nodes-1)
        
        tmpCascade = sess.run(build_cascade(tau, seed, T))
        
        
        order = tmpCascade.argsort()
        times = tmpCascade[order]
        
        cascadeList=[]
        
        for i in range(nodes):
            if times[i]>=T: break
            cascadeList.append(float(order[i]))
            cascadeList.append(times[i])

        cascade.append(cascadeList)
        
    return np_topics, cascade

In [49]:
test_topics, test_cascade = sampleCascade(alpha,10,200)

array([1., 0.])

# Run BFGS on Koenecker Data

In [28]:
numNodes=10

In [29]:
def infectedCascade(cascade, N=numNodes, T=10):
    inf = np.zeros((N,N))
    
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]


    for i in range(len(c_nodes)):
        for j in range(i):
            if cascade[j] < T:
                inf[(c_nodes[i],c_nodes[j])]=c_times[i]-c_times[j]
    
    return tf.convert_to_tensor(inf)

In [30]:
def uninfectedCascade(cascade,N=numNodes,T=10):
    nodes = {s for s in range(N)}
    uninf = np.zeros((N,N))

    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    
    for i in range(len(c_nodes)):
        for j in (nodes-set(c_nodes)):
            uninf[c_nodes[i],j]=T-c_times[i]

    return tf.convert_to_tensor(uninf)

In [31]:
def genInfectedTensor(v):
    tf_infected = None
    for cascade in v:
        if tf_infected == None:
            tf_infected = tf.expand_dims(infectedCascade(cascade),0)
        else:
            tf_infected = tf.concat([tf_infected,tf.expand_dims(infectedCascade(cascade),0)],axis=0)
    return tf_infected

In [32]:
def genUninfectedTensor(v):
    tf_uninfected = None
    for cascade in v:
        if tf_uninfected == None:
            tf_uninfected = tf.expand_dims(uninfectedCascade(cascade),0)
        else:
            tf_uninfected = tf.concat([tf_uninfected,tf.expand_dims(uninfectedCascade(cascade),0)],axis=0)
    return tf_uninfected

In [33]:
I = genInfectedTensor(test_cascade)

In [34]:
U = genUninfectedTensor(test_cascade)

In [35]:
topics = tf.convert_to_tensor(test_topics,dtype=tf.float32)

In [36]:
U_ph = tf.placeholder(tf.float32, U.shape)
I_ph = tf.placeholder(tf.float32, I.shape)

In [51]:
numTopics = 2
theta_topics = tf.reshape(tf.divide(tf.ones((1,numTopics)),numTopics),(numTopics,1))

array([[0.5],
       [0.5]], dtype=float32)

In [38]:
rate_intercept = tf.get_variable('rate_intercept', initializer=tf.zeros_initializer, shape=(10,10))
rate_affinity = tf.get_variable('rate_affinity', initializer=tf.zeros_initializer, shape=(10,10,numTopics))

In [53]:
def f_psi_1(rate_intercept, rate_affinity, infected, topic):
    rate_affinity_rshp = tf.add(tf.reshape(rate_affinity,(100,2)),.001)
    topic_rshp = tf.reshape(topic,(2,1))
    affinity = tf.reshape(tf.matmul(rate_affinity_rshp,topic_rshp),(10,10))
    
    alpha_tensor = tf.add(tf.nn.relu(tf.add(rate_intercept,affinity)),.001)
    return -tf.reduce_sum(tf.multiply(alpha_tensor,tf.cast(infected,dtype=tf.float32)))

psi_1 = tf.map_fn(lambda x: f_psi_1(rate_intercept, rate_affinity, x[0], x[1]), (I_ph, topics), dtype=tf.float32)

In [54]:
def f_psi_2(rate_intercept, rate_affinity, uninfected, topic):
    rate_affinity_rshp = tf.add(tf.reshape(rate_affinity,(100,2)),.001)
    topic_rshp = tf.reshape(topic,(2,1))
    affinity = tf.reshape(tf.matmul(rate_affinity_rshp,topic_rshp),(10,10))
    
    alpha_tensor = tf.add(tf.nn.relu(tf.add(rate_intercept,affinity)),.001)
    
    return -tf.reduce_sum(tf.multiply(tf.transpose(alpha_tensor),tf.cast(uninfected,dtype=tf.float32)))

psi_2 = tf.map_fn(lambda x: f_psi_2(rate_intercept, rate_affinity, x[0], x[1]), (U_ph, topics), dtype=tf.float32)


In [55]:
def f_psi_3(rate_intercept, rate_affinity, infected, topic):
    rate_affinity_rshp = tf.add(tf.reshape(rate_affinity,(100,2)),.001)
    topic_rshp = tf.reshape(topic,(2,1))
    alpha_tensor = tf.add(tf.nn.relu(tf.add(rate_intercept,tf.reshape(tf.matmul(rate_affinity_rshp,topic_rshp),(10,10)))),.001)
    
    infected_sign = tf.cast(tf.sign(infected),tf.float32)
    
    # Row sum infected
    alpha_tensor_row = tf.reduce_sum(tf.multiply(infected_sign,alpha_tensor),axis=1)
    
    # Add 1 to 0 entries so log(1)=0
    alpha_tensor_row_zeros = -tf.cast(tf.sign(alpha_tensor_row),tf.float32)+1
    
    return tf.reduce_sum(tf.log(tf.add(alpha_tensor_row,alpha_tensor_row_zeros)))

psi_3 = tf.map_fn(lambda x: f_psi_3(rate_intercept, rate_affinity, x[0], x[1]), (I_ph, topics), dtype=tf.float32)

In [77]:
def gamma_prior(rate_intercept, rate_affinity, theta_topics):
    rate_affinity_rshp = tf.add(tf.reshape(rate_affinity,(100,2)),.001)
    
    return -tf.add(tf.nn.relu(rate_intercept+tf.reshape(tf.matmul(rate_affinity_rshp,theta_topics),(10,10))),.001)

prior = gamma_prior(rate_intercept, rate_affinity,theta_topics)

In [78]:
log_p = -(tf.reduce_sum(prior) + tf.reduce_sum(psi_1)+tf.reduce_sum(psi_2)+tf.reduce_sum(psi_3))

In [79]:
max_iter = 2000

data = {U_ph: U.eval(session=sess),
        I_ph: I.eval(session=sess)}

optimizer = tf.contrib.opt.ScipyOptimizerInterface(log_p, 
                                                   method='L-BFGS-B',
                                                   options={'maxiter': max_iter})
model = tf.global_variables_initializer()
sess = tf.Session()

sess.run(model)
optimizer.minimize(sess, feed_dict=data)
# a = alpha_tensor.eval(session=sess)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1022.493530
  Number of iterations: 27
  Number of functions evaluations: 74


In [80]:
def evaluateAlpha(rate_intercept, rate_affinity, topic):
    rate_affinity_rshp = tf.add(tf.reshape(rate_affinity,(100,2)),.001)
    topic_rshp = tf.convert_to_tensor(np.reshape(topic,(2,1)),dtype=tf.float32)
    alpha_tensor = tf.add(tf.nn.relu(tf.add(rate_intercept,tf.reshape(tf.matmul(rate_affinity_rshp,topic_rshp),(10,10)))),.001)
    
    negative_I = 1-tf.eye(10)
    
    return tf.multiply(alpha_tensor,negative_I)

In [81]:
infer_alpha_0 = evaluateAlpha(rate_intercept,rate_affinity,np.array([1,0])).eval(session=sess).transpose().round(1)
infer_alpha_1 = evaluateAlpha(rate_intercept,rate_affinity,np.array([0,1])).eval(session=sess).transpose().round(1)

In [82]:
infer_alpha_0

array([[0. , 0.9, 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1.2, 0.2, 0. , 0. , 1.7, 0.3, 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0.1, 0. , 0.3, 0. ],
       [0. , 0. , 0. , 0. , 0. , 1.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1.1, 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)

In [83]:
alpha_0.round(1)

array([[0. , 1. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0.3, 0. , 0. , 2. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0.1, 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)

In [84]:
infer_alpha_1

array([[0. , 0.7, 0. , 0. , 0.7, 0. , 0. , 0.2, 0. , 0. ],
       [0. , 0. , 1.3, 0.2, 0. , 0. , 1.7, 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0.8, 0. , 0. , 0. , 0.6, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0.6, 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1.4, 0. , 0.6, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.8, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)

In [85]:
alpha_1.round(1)

array([[0. , 1. , 0. , 0.3, 0.8, 0. , 0. , 1.2, 0. , 0. ],
       [0. , 0. , 1.4, 0.3, 0. , 0. , 3.5, 0.6, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0.7, 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.8, 0. , 0. , 1.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1.4, 0. , 0.6, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.7, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)

# Criticize Results

In [ ]:
def printCascade(cascade):
    print("order\t node\t time")
    print("-----\t ----\t ----")
    for i in range(len(cascade)//2):
        print('{:5d}\t {:4d}\t {:0.2f}'.format(i+1,int(cascade[i*2]), cascade[i*2+1]))

printCascade(v[0])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
optimizer._var_updates

In [ ]:
alpha = sess.run(optimizer._vars)[0]
beta = sess.run(optimizer._vars)[1]
tf.nn.relu(beta[753][261])

In [ ]:
from tensorflow import tensorflow_probability
# 
# .Beta(alpha, beta)